In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
eddir = '/home/disk/eos8/ed/cmip6/siconc_day_historical/'

In [3]:
allfiles = sorted([f for f in os.listdir(eddir) if '.nc' in f])

In [4]:
proceddir = '/home/disk/eos8/ed/cmip6/proc_si/'

In [5]:
epfiles = sorted([f for f in os.listdir(proceddir) if '.mat' in f and 'CESM1' not in f])
epfiles[0]

'sia_sie_ACCESS-CM2_19500101_20100101.mat'

In [6]:
emodels = sorted(list(set([f.split('_')[2] for f in epfiles])))

In [7]:
emodels

['ACCESS-CM2',
 'BCC-CSM2-MR',
 'BCC-ESM1',
 'CESM2',
 'CESM2-WACCM',
 'CNRM-CM6-1',
 'CNRM-ESM2-1',
 'CanESM5',
 'EC-Earth3',
 'EC-Earth3-Veg',
 'GFDL-CM4',
 'IPSL-CM6A-LR',
 'MIROC6',
 'MPI-ESM-1-2-HAM',
 'MPI-ESM1-2-HR',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NorESM2-LM',
 'NorESM2-MM',
 'SAM0-UNICON']

In [8]:
gridfiles = os.listdir('GRIDS')

In [10]:
for model in emodels[:]:
    if model!='GFDL-CM4':
        print(model)
        myfiles = sorted([eddir+f for f in allfiles if model+'_' in f and 'r1i' in f])
        gridfname = [f for f in gridfiles if model+'_' in f][0]
        ens = myfiles[0].split('_')[6]

        myfiles = [f for f in myfiles if ens in f]
        ds = xr.open_mfdataset(myfiles).sel(time=slice('1978','2014'))

        gds = xr.open_dataset('GRIDS/'+gridfname)
        if 'CESM' in model: #CESM2 and SAM0 have 1e30 in latitude
            gds = gds.rename({'nlon':'ni','nlat':'nj'})


        mylat = [f for f in gds.coords if 'lat' in f][-1]
        mylon = [f for f in gds.coords if 'lon' in f][-1]

        gds[mylat]=ds[mylat]
        gds[mylon]=ds[mylon]
        mydims = [f for f in gds.areacello.dims]

        ints = ds.siconc.copy()
        ints = ints.where(ints>15.)
        ints = ints.where(ints!=ints,1.)

        sia_nh = ((ds.siconc/100.*gds.areacello).where(ds[mylat]>0.).sum(dim=mydims)/1e12).to_dataset(name='sia_nh')
        sie_nh = ((ints*gds.areacello).where(ds[mylat]>0.).sum(dim=mydims)/1e12).to_dataset(name='sie_nh')
        sia_sh = ((ds.siconc/100.*gds.areacello).where(ds[mylat]<0.).sum(dim=mydims)/1e12).to_dataset(name='sia_sh')
        sie_sh = ((ints*gds.areacello).where(ds[mylat]<0.).sum(dim=mydims)/1e12).to_dataset(name='sie_sh')

        outds = xr.merge([sia_nh, sie_nh, sia_sh, sie_sh])

        outds.attrs = ds.attrs
        outds.sia_nh.attrs['units'] = '10^6 km^2'
        outds.sie_nh.attrs['units'] = '10^6 km^2'
        outds.sia_sh.attrs['units'] = '10^6 km^2'
        outds.sie_sh.attrs['units'] = '10^6 km^2'

        outds.attrs['processed'] = 'Lettie Roach, 2021'
        outds.to_netcdf('/home/disk/sipn/lroach/cmip6_dailysia/sia_sie_SIday_historical_'+model+'_'+ens+'_'+myfiles[0].split('_')[7]+'_1978-2014.nc')


ACCESS-CM2
BCC-CSM2-MR
BCC-ESM1
CESM2


/home/disk/atmos/lroach/anaconda3/lib/python3.8/site-packages/xarray/conventions.py:492: SerializationWarning: variable 'siconc' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/disk/atmos/lroach/anaconda3/lib/python3.8/site-packages/xarray/conventions.py:492: SerializationWarning: variable 'areacello' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


CESM2-WACCM


/home/disk/atmos/lroach/anaconda3/lib/python3.8/site-packages/xarray/conventions.py:492: SerializationWarning: variable 'siconc' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/disk/atmos/lroach/anaconda3/lib/python3.8/site-packages/xarray/conventions.py:492: SerializationWarning: variable 'areacello' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


CNRM-CM6-1
CNRM-ESM2-1
CanESM5
EC-Earth3
EC-Earth3-Veg
IPSL-CM6A-LR
MIROC6
MPI-ESM-1-2-HAM
MPI-ESM1-2-HR
MPI-ESM1-2-LR
MRI-ESM2-0
NorESM2-LM
NorESM2-MM
SAM0-UNICON


In [11]:
sia

NameError: name 'sia' is not defined

In [12]:
#GDFL grid

In [13]:
for model in ['GFDL-CM4']:
    print(model)
    myfiles = sorted([eddir+f for f in allfiles if model+'_' in f and 'r1i' in f])
    gridfname = [f for f in gridfiles if model+'_' in f][0]
    ens = myfiles[0].split('_')[6]

    myfiles = [f for f in myfiles if ens in f]
    ds = xr.open_mfdataset(myfiles).sel(time=slice('1978','2014'))
    
    gds = xr.open_dataset('GRIDS/areacella_fx_GISS-E2-1-G-CC_piControl_r1i1p1f1_gn.nc')
    if 'CESM' in model: #CESM2 and SAM0 have 1e30 in latitude
        gds = gds.rename({'nlon':'ni','nlat':'nj'})
           

    mylat = [f for f in gds.coords if 'lat' in f][-1]
    mylon = [f for f in gds.coords if 'lon' in f][-1]
    
    gds[mylat]=ds[mylat]
    gds[mylon]=ds[mylon]
    mydims = [f for f in gds.areacella.dims]
    
    ints = ds.siconc.copy()
    ints = ints.where(ints>15.)
    ints = ints.where(ints!=ints,1.)
     
    sia_nh = ((ds.siconc/100.*gds.areacella).where(ds[mylat]>0.).sum(dim=mydims)/1e12).to_dataset(name='sia_nh')
    sie_nh = ((ints*gds.areacella).where(ds[mylat]>0.).sum(dim=mydims)/1e12).to_dataset(name='sie_nh')
    sia_sh = ((ds.siconc/100.*gds.areacella).where(ds[mylat]<0.).sum(dim=mydims)/1e12).to_dataset(name='sia_sh')
    sie_sh = ((ints*gds.areacella).where(ds[mylat]<0.).sum(dim=mydims)/1e12).to_dataset(name='sie_sh')
    
    outds = xr.merge([sia_nh, sie_nh, sia_sh, sie_sh])
    
    outds.attrs = ds.attrs
    outds.sia_nh.attrs['units'] = '10^6 km^2'
    outds.sie_nh.attrs['units'] = '10^6 km^2'
    outds.sia_sh.attrs['units'] = '10^6 km^2'
    outds.sie_sh.attrs['units'] = '10^6 km^2'
    
    outds.attrs['processed'] = 'Lettie Roach, 2021'
    outds.to_netcdf('/home/disk/sipn/lroach/cmip6_dailysia/sia_sie_SIday_historical_'+model+'_'+ens+'_'+myfiles[0].split('_')[7]+'_1978-2014.nc')


GFDL-CM4
